# Viewing angles

Visualizing the illumination and viewing angles. Explaining how ViSiProg groups are related?

In [45]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import numpy as np
import os

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import plotly.graph_objs as go
from visiprog.data import read_VSP_label, read_img_list, read_viewing_conditions, viewing_condition_index
from visiprog.data import read_viewing_conditions, polar_to_euclidean, angle_between
from visiprog.data import illum_spatial_adjacent_graph, viewing_spatial_adjacent_graph
from IPython.display import display, HTML

from scipy.spatial import Voronoi, SphericalVoronoi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
df_viewing = read_viewing_conditions()

np.savetxt('df_viewing.csv', df_viewing, fmt='%.4f', delimiter=',')

print(df_viewing.columns.values)
print(df_viewing.shape)

['view_theta' 'view_phi' 'illum_theta' 'illum_phi' 'illum_x' 'illum_y'
 'illum_z' 'view_x' 'view_y' 'view_z']
(205, 10)


In [25]:
def make_image_html(image_path):
     return '<img src="{}" width="100" height="100" style="display:inline;margin:1px"/>'.format(image_path)

In [36]:
groups = read_VSP_label()
list_img = read_img_list()

traces = []

group = groups[169]

for i in group: 
    viewing_index = viewing_condition_index(list_img[i])
    
    illum_x = df_viewing['illum_x'][viewing_index]
    illum_y = df_viewing['illum_y'][viewing_index]
    illum_z = df_viewing['illum_z'][viewing_index]

    view_x = df_viewing['view_x'][viewing_index]
    view_y = df_viewing['view_y'][viewing_index]
    view_z = df_viewing['view_z'][viewing_index]
    
    line_x = [illum_x, illum_x, 0, view_x]
    line_y = [illum_y - 0.05, illum_y, 0, view_y]
    line_z = [illum_z,illum_z, 0, view_z]
    
    # plotting line
    line_scatter = go.Scatter3d(
        x=line_x,
        y=line_y,
        z=line_z,
        name=os.path.basename(list_img[i])
    )
    
    traces.append(line_scatter)

layout = go.Layout(
    title='Angles of VSP group',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename='vspgroup')

# display the images as well
display(HTML(''.join(make_image_html(list_img[i]) for i in group) + '<br>' + \
            '\t'.join(os.path.basename(list_img[i]) for i in group)))

# Formal analysis using graph and minimum spanning tree

In [33]:
groups = read_VSP_label(pappas_only=True, sorted_by_material=True)
list_img = read_img_list()

print(len(groups))

mst_distances = []

for group in groups:
    
    # building the weighted undirected graph
    G = nx.Graph()
    G.add_nodes_from(range(len(group)))

    for i in range(len(group)):
        index_i = viewing_condition_index(list_img[group[i]])
        
        illum_x_1 = df_viewing['illum_x'][index_i]
        illum_y_1 = df_viewing['illum_y'][index_i]
        illum_z_1 = df_viewing['illum_z'][index_i]
        
        view_x_1 = df_viewing['view_x'][index_i]
        view_y_1 = df_viewing['view_y'][index_i]
        view_z_1 = df_viewing['view_z'][index_i]
        
        for j in range(i + 1, len(group)):
            index_j = viewing_condition_index(list_img[group[j]])
            illum_x_2, illum_y_2, illum_z_2 = polar_to_euclidean(df_viewing['illum_theta'][index_j],df_viewing['illum_phi'][index_j])
            view_x_2, view_y_2, view_z_2 = polar_to_euclidean(df_viewing['view_theta'][index_j],df_viewing['view_phi'][index_j])
            
            weight_illum = angle_between((illum_x_1, illum_y_1, illum_z_1),(illum_x_2, illum_y_2, illum_z_2))
            weight_view = angle_between((view_x_1, view_y_1, view_z_1),(view_x_2, view_y_2, view_z_2))
        
            G.add_edge(i, j, weight=weight_illum)
            G.add_edge(i, j, weight=weight_view)
            
#     print(G.edges(data=True))
#     print("\n\nSorting the MST\n\n")
    
    # minimum spanning tree
    mst=nx.minimum_spanning_edges(G,data=True)
    
    distances = []
    for e in mst:
        distances.append(e[2]['weight'])
        
    mst_distances.append(distances)

456


In [20]:
mst_out = np.asarray(mst_distances)
np.savetxt('mst_distances.csv', mst_out, fmt='%.4f', delimiter=',')
print(mst_out.shape)

(456, 8)


(604, 8)


# Formal analysis using graph and VORONOI DIAGRAM

Why? Since angle does not explain a lot. It could be better explained through spatial adjacency.

In [46]:
# building spatial graph for illumination angles
G_illum = illum_spatial_adjacent_graph()
# viewing_edges = viewing_spatial_adjacent_graph()


print(G_illum.edges())

print(nx.is_connected(G_illum))

{(1.1502620000000001, -0.684719): Int64Index([16, 69, 115, 153, 180], dtype='int64'), (0.955317, -1.136277): Int64Index([25, 75, 120, 157], dtype='int64'), (0.955317, -0.15348399999999998): Int64Index([26, 76, 121, 158, 184, 201], dtype='int64'), (0.39269899999999996, -1.5707959999999999): Int64Index([48, 96, 135], dtype='int64'), (1.154239, -1.0454): Int64Index([13, 67, 113, 151], dtype='int64'), (0.955317, -2.988109): Int64Index([23], dtype='int64'), (0.662145, -2.005315): Int64Index([35, 83, 126], dtype='int64'), (0.876815, -0.40305599999999997): Int64Index([30, 79, 123, 160, 186], dtype='int64'), (0.981748, -1.5707959999999999): Int64Index([21, 72, 118], dtype='int64'), (1.357866, -0.15348399999999998): Int64Index([9, 63, 110, 149, 177, 195, 205], dtype='int64'), (0.5890479999999999, 3.141593): Int64Index([40, 88], dtype='int64'), (0.662145, -2.860557): Int64Index([34, 82], dtype='int64'), (1.370707, -2.3767650000000002): Int64Index([3], dtype='int64'), (1.360401, -0.09462899999999

TypeError: object of type 'float' has no len()

In [85]:
G_illum = nx.Graph()
G_illum.add_nodes_from(range(df_viewing.shape[0]))

for e in illum_edges:
    G_illum.add_edge(e[0], e[1])

print(nx.is_connected(G_illum))

print(G_illum.edges())

G_viewing = nx.Graph()
G_viewing.add_nodes_from(range(df_viewing.shape[0]))

for e in viewing_edges:
    G_viewing.add_edge(e[0], e[1])

print(nx.is_connected(G_viewing))

False
[(0, 65), (0, 2), (0, 3), (0, 12), (0, 9), (1, 10), (1, 4), (2, 17), (2, 14), (2, 65), (3, 145), (3, 12), (3, 15), (4, 6), (4, 7), (4, 8), (4, 145), (4, 10), (6, 8), (6, 10), (6, 13), (7, 16), (7, 145), (7, 116), (7, 8), (8, 16), (8, 13), (9, 65), (9, 20), (9, 24), (9, 73), (9, 12), (10, 21), (10, 13), (12, 121), (12, 24), (12, 15), (13, 16), (13, 21), (13, 75), (14, 17), (14, 28), (14, 26), (14, 65), (15, 145), (15, 116), (15, 121), (15, 78), (16, 75), (16, 116), (17, 28), (17, 22), (20, 24), (20, 73), (20, 79), (21, 80), (21, 75), (22, 28), (22, 30), (22, 81), (24, 161), (24, 121), (24, 79), (26, 65), (26, 82), (26, 37), (26, 73), (26, 28), (28, 37), (28, 81), (30, 39), (37, 81), (37, 82), (37, 49), (37, 42), (37, 91), (37, 94), (39, 81), (39, 94), (40, 161), (40, 82), (40, 95), (40, 79), (41, 80), (41, 84), (41, 135), (42, 81), (42, 99), (42, 49), (45, 50), (45, 84), (45, 53), (45, 86), (45, 135), (49, 99), (49, 52), (49, 54), (49, 91), (50, 52), (50, 54), (50, 86), (50, 132),

In [69]:
nx.shortest_path_length(G_illum,source = 0, target=2)

1

In [78]:
groups, _ = read_VSP_label(pappas_only=True, sorted_by_material=True)
list_img = read_img_list()


mst_spatial_distances = []

for group in groups:    
    # building the weighted undirected graph
    G = nx.Graph()
    G.add_nodes_from(range(len(group)))

    for i in range(len(group)):
        m_i = viewing_condition_index(list_img[group[i]])
        for j in range(i + 1, len(group)):
            m_j = viewing_condition_index(list_img[group[j]])
            print(m_i, m_j)
            weight_illum = nx.shortest_path_length(G_illum, source=m_i - 1, target=m_j - 1)
            weight_view = nx.shortest_path_length(G_viewing, source=m_i - 1, target=m_j - 1)
            
            print(weight_illum)
            
            G.add_edge(i, j, weight = max(weight_illum, weight_view))
            
    distances = []
    for e in mst:
        distances.append(e[2]['weight'])
        
    mst_spatial_distances.append(distances)

175 149


NetworkXNoPath: No path between 174 and 148.